In [ ]:
%%bigquery
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False
LIMIT 5

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_lm
OPTIONS(input_label_cols=['ontime'],
        model_type='logistic_reg',
        data_split_method='custom',
        data_split_col='is_eval_day')
AS
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False

In [ ]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL dsongcp.arr_delay_lm)

In [ ]:
###########################################################################################################

In [ ]:
%%bash
echo Task 2 Completed

In [ ]:
%%bigquery
SELECT *
FROM ML.EVALUATE(MODEL dsongcp.arr_delay_lm,
                 (
                     
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
    is_train_day = 'False'
                     
                 ),
                 STRUCT(0.7 AS threshold))

In [ ]:
%%bigquery
SELECT * FROM ML.WEIGHTS(MODEL dsongcp.arr_delay_lm)

In [ ]:
%%bigquery
SELECT * FROM ML.PREDICT(MODEL dsongcp.arr_delay_lm,
                        (
SELECT 12.0 AS dep_delay, 14.0 AS taxi_out, 1231 AS distance
                        ))

In [ ]:
%%bigquery
WITH predictions AS (
SELECT
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),
stats AS (
SELECT
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)
SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_airports_lm
OPTIONS(input_label_cols=['ontime'],
        model_type='logistic_reg',
        data_split_method='custom',
        data_split_col='is_eval_day')
AS
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False

In [ ]:
###########################################################################################################

In [ ]:
%%bash
echo Task 3 completed

In [ ]:
%%bigquery
SELECT *
FROM ML.EVALUATE(MODEL dsongcp.arr_delay_airports_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))

In [ ]:
%%bigquery
WITH predictions AS (
SELECT
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_airports_lm,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),
stats AS (
SELECT
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)
SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats